In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers import Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

In [12]:
# these are labels and also the exact folder names in which each kind of images are

view_map = {
    'buildings': 0,
    'forest': 1,
    'glacier': 2,
    'mountain': 3,
    'sea': 4,
    'street': 5,
}

In [13]:
fpath=r"/content/drive/MyDrive/Image Classification/Image Classification"          # main parent folder containing train and test

frames = []

for p in ['seg_train', 'seg_test']:  # train and test folder names
    for k in view_map.keys():  # use your own label map
        fdf = pd.DataFrame(columns=['view', 'pixels', 'height', 'width', 'usage'])  # DataFrame for image data
        folder_path = os.path.join(fpath, p, p, k)  # Update the path
        all_files = os.listdir(folder_path)  # List all files in the directory

        for i, file in enumerate(all_files):
            pic_path = os.path.join(folder_path, file)
            try:
                img = cv2.imread(pic_path)
                if img is None:
                    print(f"Error: Unable to read image '{pic_path}'")
                    continue
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                height, width = img.shape
                img = list(img.flatten())
                img = str(img).replace(',', '')[1: -1].strip()

                fdf.loc[i] = [view_map[k], img, height, width, p]
            except Exception as e:
                print(f"Error occurred while processing '{pic_path}': {e}")

        frames.append(fdf)
        print(len(frames))

1
2
3
4
5
6
7
8
9
10
11
12


In [3]:
image_df = pd.read_csv(r'C:\Users\SHALAKA\Downloads\imageclassification.csv')
image_df

,view,pixels,height,width,usage
0,0,247 247 248 248 249 249 249 249 248 249 250 25...,150,150,seg_train
1,0,113 114 113 111 111 113 114 112 113 112 111 11...,150,150,seg_train
2,0,181 181 181 181 181 181 181 181 182 182 185 17...,150,150,seg_train
3,0,196 168 171 167 169 171 167 161 162 170 171 17...,150,150,seg_train
4,0,26 26 27 27 27 27 27 27 27 27 26 26 26 27 28 2...,150,150,seg_train
...,...,...,...,...,...
17029,5,110 115 80 127 213 165 172 179 47 102 81 196 2...,150,150,seg_test
17030,5,50 58 66 74 83 91 91 87 98 104 93 147 255 251 ...,150,150,seg_test
17031,5,69 69 69 69 69 69 70 70 70 70 70 70 70 71 71 7...,150,150,seg_test
17032,5,150 152 153 151 151 153 154 152 154 154 154 15...,150,150,seg_test


In [4]:
def pixel_convert(df, chunk_size=100):

    new_df = df.copy()

    num_chunks = len(new_df) // chunk_size + 1
    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len(new_df))
        df_chunk = new_df.iloc[start_idx:end_idx].copy()


        df_chunk['pixels'] = df_chunk['pixels'].apply(lambda x: x.split())
        df_chunk['pixels'] = df_chunk['pixels'].apply(lambda x: [int(pixel) for pixel in x])
        df_chunk['pixels'] = df_chunk.apply(lambda row: np.array(row['pixels']).reshape(row['height'], row['width']), axis=1)

        new_df.iloc[start_idx:end_idx] = df_chunk

    return new_df



In [5]:
converted_df = pixel_convert(image_df)
converted_df

,view,pixels,height,width,usage
0,0,"[[247, 247, 248, 248, 249, 249, 249, 249, 248,...",150,150,seg_train
1,0,"[[113, 114, 113, 111, 111, 113, 114, 112, 113,...",150,150,seg_train
2,0,"[[181, 181, 181, 181, 181, 181, 181, 181, 182,...",150,150,seg_train
3,0,"[[196, 168, 171, 167, 169, 171, 167, 161, 162,...",150,150,seg_train
4,0,"[[26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 26, ...",150,150,seg_train
...,...,...,...,...,...
17029,5,"[[110, 115, 80, 127, 213, 165, 172, 179, 47, 1...",150,150,seg_test
17030,5,"[[50, 58, 66, 74, 83, 91, 91, 87, 98, 104, 93,...",150,150,seg_test
17031,5,"[[69, 69, 69, 69, 69, 69, 70, 70, 70, 70, 70, ...",150,150,seg_test
17032,5,"[[150, 152, 153, 151, 151, 153, 154, 152, 154,...",150,150,seg_test


In [6]:
def apply_zero_padding(pixel_array, original_height, original_width, target_height, target_width):

    img = np.array(pixel_array).reshape(original_height, original_width)


    pad_height = max(0, target_height - original_height)
    pad_width = max(0, target_width - original_width)


    top_pad = pad_height // 2
    bottom_pad = pad_height - top_pad
    left_pad = pad_width // 2
    right_pad = pad_width - left_pad


    padded_img = cv2.copyMakeBorder(img, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=0)

    return padded_img

max_height = converted_df['height'].max()
max_width = converted_df['width'].max()


converted_df['pixels'] = converted_df.apply(lambda row: apply_zero_padding(row['pixels'], row['height'], row['width'], max_height, max_width), axis=1)


**Train Test Split**

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming your DataFrame is named converted_df
# Splitting the DataFrame into train, test, and validation sets based on the 'usage' column
train_df = converted_df[converted_df['usage'] == 'seg_train']
test_df = converted_df[converted_df['usage'] == 'seg_test']

# Further splitting the train data into train and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Print shapes of the datasets (optional)
print(f"Train Data Shape: {train_data.shape}")
print(f"Validation Data Shape: {val_data.shape}")
print(f"Test Data Shape: {test_df.shape}")

Train Data Shape: (11227, 5)
Validation Data Shape: (2807, 5)
Test Data Shape: (3000, 5)


In [8]:
def preprocess_images(data):
    # Convert the 'pixels' column to numpy array and normalize pixel values
    images = np.array(data['pixels'].tolist()) / 255.0

    # Reshape the images to the required shape for CNN (assuming grayscale images)
    images = images.reshape(-1, 150, 150, 1)  # Adjust dimensions as needed
    return images

# Preprocess the images for training, validation, and test sets
train_images = preprocess_images(train_data)
val_images = preprocess_images(val_data)
test_images = preprocess_images(test_df)

In [21]:
model = Sequential()

# Add Convolutional layers
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 1), strides=(1, 1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),  strides=(1, 1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3),  strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add Dense layers
#model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(6, activation='softmax'))

# Compile the model
#learning_rate = 0.00001

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-5,
    decay_steps=2000,  
    decay_rate=0.8  )

optimizer = Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



# Train the model using your training data and validate it using validation data
history = model.fit(train_images, train_data['view'], 
                    epochs=80, 
                    validation_data=(val_images, val_data['view']))


Epoch 1/80
351/351 [==============================] - 80s 220ms/step - loss: 1.5773 - accuracy: 0.3854 - val_loss: 2.1643 - val_accuracy: 0.2027
Epoch 2/80
351/351 [==============================] - 74s 211ms/step - loss: 1.2294 - accuracy: 0.5122 - val_loss: 1.0855 - val_accuracy: 0.5789
Epoch 3/80
351/351 [==============================] - 75s 214ms/step - loss: 1.1195 - accuracy: 0.5640 - val_loss: 0.9708 - val_accuracy: 0.6213
Epoch 4/80
351/351 [==============================] - 75s 214ms/step - loss: 1.0453 - accuracy: 0.5992 - val_loss: 0.9179 - val_accuracy: 0.6437
Epoch 5/80
351/351 [==============================] - 76s 216ms/step - loss: 0.9865 - accuracy: 0.6239 - val_loss: 0.8765 - val_accuracy: 0.6580
Epoch 6/80
351/351 [==============================] - 76s 218ms/step - loss: 0.9394 - accuracy: 0.6418 - val_loss: 0.8436 - val_accuracy: 0.6772
Epoch 7/80
351/351 [==============================] - 78s 223ms/step - loss: 0.8864 - accuracy: 0.6679 - val_loss: 0.8061 - val_ac

In [22]:
# Evaluate the model using the test set
test_loss, test_acc = model.evaluate(test_images, test_df['view'])
print(f'Test accuracy: {test_acc}')

94/94 [==============================] - 6s 58ms/step - loss: 0.5947 - accuracy: 0.7833
Test accuracy: 0.7833333611488342


In [23]:

# Save the entire model
model.save(r'C:\Users\SHALAKA\Desktop\MachineL\my_model')

# Load the model
loaded_model = tf.keras.models.load_model('my_model')

INFO:tensorflow:Assets written to: C:\Users\SHALAKA\Desktop\MachineL\my_model\assets


INFO:tensorflow:Assets written to: C:\Users\SHALAKA\Desktop\MachineL\my_model\assets
